In [4]:
!pip install dython

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile
import os
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, OneHotEncoder,LabelEncoder, LabelBinarizer
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tqdm import tqdm
import matplotlib.pyplot as plt 
import numpy as np 
import math
from dython import nominal
from datetime import datetime

In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        file_zip_path = os.path.join(dirname, filename)
        print(file_zip_path)
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
train_users_2 = pd.read_csv('/kaggle/input/airbnb-recruiting-new-user-bookings/train_users_2.csv.zip')
countries = pd.read_csv('/kaggle/input/airbnb-recruiting-new-user-bookings/countries.csv.zip')
age_gender_bkts = pd.read_csv('/kaggle/input/airbnb-recruiting-new-user-bookings/age_gender_bkts.csv.zip')
test_users = pd.read_csv('/kaggle/input/airbnb-recruiting-new-user-bookings/test_users.csv.zip')
sessions = pd.read_csv('/kaggle/input/airbnb-recruiting-new-user-bookings/sessions.csv.zip')

train_users_2

## Unique values

In [8]:
def verify_unique_values(table):
    for col in table.columns:
        print(f"\n{col} ({len(table[col].unique())}): ", table[col].unique(), "\n")
        if col != "id" and col != "date_account_created" and col != "timestamp_first_active" and col != "date_first_booking":
            for only in table[col].unique():
                print(f"{only}:", list(table[col]).count(only))
                
verify_unique_values(train_users_2)

## Verificar quantidade de valores NAN 

In [9]:
print("TOTAL:", len(train_users_2["date_first_booking"]), "amostras")
print(train_users_2.isna().sum())
print()
print(test_users.isna().sum())

# Dropar colunas

In [10]:
train_cols_dropped = train_users_2.drop("date_first_booking", axis="columns")
#train_cols_dropped = train_cols_dropped.drop("id", axis="columns")
#train_cols_dropped = train_cols_dropped.drop("age", axis="columns")
train_cols_dropped = train_cols_dropped.drop("date_account_created", axis="columns")
train_cols_dropped = train_cols_dropped.drop("timestamp_first_active", axis="columns")
print(train_cols_dropped.columns)

test_users_id = test_users.id.copy()
test_cols_dropped = test_users.drop("date_first_booking", axis="columns")
#test_cols_dropped = test_cols_dropped.drop("id", axis="columns")
#test_cols_dropped = test_cols_dropped.drop("age", axis="columns")
test_cols_dropped = test_cols_dropped.drop("date_account_created", axis="columns")
test_cols_dropped = test_cols_dropped.drop("timestamp_first_active", axis="columns")
print(test_cols_dropped.columns)

# Corrigindo idades de "age" no formato errado

In [11]:
train_age_corrected = train_cols_dropped.copy()
train_age_corrected.loc[(train_age_corrected["age"] > 1900) & (train_age_corrected["age"] < 2000), "age"] = 2015 - train_age_corrected["age"]
train_age_corrected = train_age_corrected.drop(train_age_corrected.loc[train_age_corrected["age"] > 90].index, axis=0)
train_age_corrected = train_age_corrected.drop(train_age_corrected.loc[train_age_corrected["age"] < 10].index, axis=0)
print(f"ages droppeds train = {len(train_cols_dropped) - len(train_age_corrected)}")

test_age_corrected = test_cols_dropped.copy()
test_age_corrected.loc[(test_age_corrected["age"] > 1900) & (test_age_corrected["age"] < 2000), "age"] = 2015 - test_age_corrected["age"]
test_age_corrected = test_age_corrected.drop(test_age_corrected.loc[test_age_corrected["age"] > 90].index, axis=0)
test_age_corrected = test_age_corrected.drop(test_age_corrected.loc[test_age_corrected["age"] < 10].index, axis=0)
print(f"ages droppeds test = {len(test_cols_dropped) - len(test_age_corrected)}")

## Preenchendo valores NAN de "age" com a mediana 

In [12]:
print("Before:")

print(test_age_corrected["age"].value_counts())
print(train_age_corrected["age"].value_counts())

test_age_corrected["age"] = test_age_corrected["age"].fillna(round(train_age_corrected["age"].median()))
train_age_corrected["age"] = train_age_corrected["age"].fillna(round(train_age_corrected["age"].median()))

print("After:")
print(test_age_corrected["age"].value_counts())
print(train_age_corrected["age"].value_counts())

# Tirar valores únicos com pouca quantidade
- Variável **first_browser**

In [13]:
def change_value_below_counts(table, column, value_to_change, counts):
    """
    Change all categorical values to "value_to_change", if their counts 
    in the column are below "counts" variable.
    """
    for value in table[column].unique(): 
        value_counts = len(table.loc[table[column] == value, column])
        if value_counts < counts:
            table.loc[table[column] == value, column] = value_to_change
    return table

train_browser_changed = change_value_below_counts(
    train_age_corrected, 
    "first_browser", 
    "-unknown-", 
    188
)
test_browser_changed = change_value_below_counts(
    test_age_corrected, 
    "first_browser", 
    "-unknown-", 
    40            # 40 because Opera has only 40 values on test set. 
)

# Corrigir NANS
- **first_affiliate_tracked** -> preencher com "untracked"

In [14]:
train_corrected_nans = train_browser_changed.copy()
train_corrected_nans.first_affiliate_tracked = train_browser_changed.first_affiliate_tracked.fillna("untracked")

test_corrected_nans = test_browser_changed.copy()
test_corrected_nans.first_affiliate_tracked = test_browser_changed.first_affiliate_tracked.fillna("untracked")

#print(train_corrected_nans.isna().sum())
print()
#print(test_corrected_nans.isna().sum())

# Analisando o dataset **sessions**

In [15]:
sessions

## Agrupando sessions de acordo com o user_id

In [16]:
sessions_grouped = sessions.groupby("user_id").agg(list)
sessions_grouped["id"] = sessions_grouped.index
sessions_grouped.index = range(len(sessions_grouped))
sessions_grouped

# Juntando sessions com dataset de treino e de teste

In [17]:
train_time_sessions = pd.merge(train_corrected_nans, sessions_grouped, how = 'left', on = 'id')
train_time_sessions.head()

In [18]:
train_time_sessions.isna().sum()

### Analisar os países de quem não tem dados no sessions

In [19]:
def plot_bar_counts_relation_target(x1, y1, x2, y2, title1, title2, figsize=(15, 5)):
    width = 0.7
    plt.subplots(figsize=figsize) 
    plt.subplot(1, 2, 1)
    plt.ylabel("counts")
    plt.xlabel("country_destination")
    plt.title(title1)
    plt.bar(x1, y1, width) 


    plt.subplot(1, 2, 2)
    plt.title(title2)
    plt.ylabel("counts")
    plt.xlabel("country_destination")
    plt.bar(x2, y2, width) 

    plt.show()

In [20]:
countries = ["NDF", "US", "other", "FR", "IT", "GB", "ES", "CA", "DE", "NL", "AU", "PT"]
counts_countries_nan_sessions = train_time_sessions.loc[train_time_sessions["action"].isna(), "country_destination"].value_counts()
counts_countries_nan_sessions = list(counts_countries_nan_sessions)
print("Quantidade sem sessions:", sum(counts_countries_nan_sessions))
print("Total:", len(train_time_sessions))
print("% Sem sessions:", sum(counts_countries_nan_sessions)/len(train_time_sessions)*100, "%")
    
counts_countries_with_sessions = train_time_sessions.loc[train_time_sessions["action"].isna() == False, "country_destination"].value_counts()

plot_bar_counts_relation_target(
    countries, 
    counts_countries_nan_sessions, 
    countries, 
    counts_countries_with_sessions,
    "Sem sessions",
    "Com sessions"
)

Portanto, não há relação especial com o alvo dos que não tem dados no sessions em comparação com os que tem. 

## Identificando relação de muitas actions com o alvo 

In [21]:
countries_many_actions = []
countries_few_actions = []

for i in range(len(train_time_sessions)):
    if not isinstance(train_time_sessions.iloc[i, 13], list): continue
    if len(train_time_sessions.iloc[i, 13]) > 200:
        countries_many_actions.append(train_time_sessions.iloc[i]["country_destination"])
    else:
        countries_few_actions.append(train_time_sessions.iloc[i]["country_destination"])
        
print(len(countries_many_actions))
print(len(countries_few_actions))

In [22]:
countries = ["NDF", "US", "other", "FR", "IT", "GB", "ES", "CA", "DE", "NL", "AU", "PT"]
count_many_actions = []
count_few_actions = []

for country in countries:
    count_many_actions.append(countries_many_actions.count(country))
    count_few_actions.append(countries_few_actions.count(country))
    
    print("Country:", country)
    print(f"% many actions: {count_many_actions[-1]/len(countries_many_actions)*100:2.3f}%")
    print(f"% few actions: {count_few_actions[-1]/len(countries_few_actions)*100:2.3f}%")
    print("\n")
    
plot_bar_counts_relation_target(
    countries, 
    count_many_actions, 
    countries, 
    count_few_actions,
    "Com muitas actions",
    'Com "poucas" actions'
)

## Criando o atributo **count_actions** para o tamanho da lista

#### Verificando se os tamanhos das listas dos atributos são os mesmos

In [23]:
# Verificando se os tamanhos das listas são os mesmos para cada atributo do sessions:  
valid = []
for i in range(len(sessions_grouped)):
    valid.append(len(sessions_grouped.iloc[i, 0]) == len(sessions_grouped.iloc[i, 1]) 
          and len(sessions_grouped.iloc[i, 0]) == len(sessions_grouped.iloc[i, 2])
          and len(sessions_grouped.iloc[i, 0]) == len(sessions_grouped.iloc[i, 3])
          and len(sessions_grouped.iloc[i, 0]) == len(sessions_grouped.iloc[i, 4])
    )
len(valid) == sum(valid)

In [24]:
sessions_grouped_count = sessions_grouped.copy()
sessions_grouped_count["count_actions"] = [np.nan]*len(sessions_grouped_count)
for i in range(len(sessions_grouped_count)):
    sessions_grouped_count.iloc[i, -1] = len(sessions_grouped_count.iloc[i, 0])

In [25]:
#sessions_grouped_count

## Tratando **secs_elapsed**

In [26]:
sessions_grouped_secs = sessions_grouped_count.copy()
sum_secs = []
for i in range(len(sessions_grouped_secs)):
    sec_list = sessions_grouped_secs.iloc[i, 4]
    total_sec = 0
    for sec in sec_list: 
        if math.isnan(sec): continue
        total_sec += sec
    sum_secs.append(total_sec)
sessions_grouped_secs["total_secs"] = sum_secs
sessions_grouped_secs = sessions_grouped_secs.drop("secs_elapsed", axis="columns")

## Transformando os valores únicos de "sessions" em colunas

In [27]:
def boolean_df(item_lists, unique_items):
    bool_dict = {}
    
    for item in tqdm(unique_items):
        try: # If it can cast to float, item is 'nan'
            float(item) 
            continue
        except:
            bool_dict[item] = item_lists.apply(lambda x: 1 if (item in x) else 0)
            
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)

def to_1D(series):
    return pd.Series([x for _list in series if isinstance(_list, list) for x in _list])

uniques = []
sessions_grouped_unique = sessions_grouped_secs.copy()
for col in sessions_grouped_unique.columns:
    if col == "id" or col == "total_secs" or col == "count_actions": continue 
    
    action_values = to_1D(sessions_grouped_unique[col])
    uniques.append(action_values.unique())
    
    print("Coluna:", col)
    print("Total values:", len(action_values))
    print("Total unique values:", len(list(action_values.unique())), "\n")
    
    actions_bool = boolean_df(sessions_grouped_unique[col], action_values.unique())
    sessions_grouped_unique = pd.concat([sessions_grouped_unique, actions_bool], axis=1)
#sessions_grouped_unique

## Identificando colunas duplicadas

In [28]:
list_sames = []
for i in tqdm(range(len(uniques))):
    for value in tqdm(uniques[i]):
        same = 0
        for j in range(len(uniques)):    
            if value in uniques[j]:
                same += 1
        if same > 1:
            list_sames.append(value)
list_sames

## Verificar se as colunas duplicadas tem os mesmos valores

In [29]:
list_sames_equal = []
list_sames_diff = []
for same in list_sames:
    if list(sessions_grouped_unique[same].iloc[:,0]) == list(sessions_grouped_unique[same].iloc[:,1]):
        list_sames_equal.append(same)
    else:
        list_sames_diff.append(same)
        
print("Total de colunas duplicadas:", len(list_sames))
print("Colunas com os mesmos valores:", len(list_sames_equal))
print("Colunas com valores diferentes:", len(list_sames_diff))

## Remover colunas com os mesmos valores

In [30]:
sessions_removed_duplicate = sessions_grouped_unique.copy()
for same_equal_col in list_sames_equal:
    for j in range(len(sessions_removed_duplicate.columns)):
        if sessions_removed_duplicate.iloc[:,j].name == same_equal_col:
            sessions_removed_duplicate = sessions_removed_duplicate.drop(sessions_removed_duplicate.columns[[j]], axis="columns")
            break
print(len(sessions_grouped_unique.columns))
print(len(sessions_removed_duplicate.columns))

## Mudar o nome do resto das colunas duplicadas

In [31]:
cols=pd.Series(sessions_removed_duplicate.columns)

for dup in cols[cols.duplicated()].unique(): 
    cols[cols[cols == dup].index.values.tolist()] = [dup + '.' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]

# rename the columns with the cols list.
sessions_removed_duplicate.columns=cols

## Dropando colunas de listas do sessions

In [32]:
sessions_removed_duplicate = sessions_removed_duplicate.drop(["action", "action_type", "action_detail", "device_type"], axis="columns")
sessions_removed_duplicate

## Eliminando colunas com poucos registros sendo 1

In [33]:
sessions_columns_verify = list(sessions_removed_duplicate.columns)
cols_dont_verify = ["id", "count_actions", "total_secs"]
for col in cols_dont_verify:
    sessions_columns_verify.remove(col)

col_few_values = []
for col in sessions_columns_verify:
    if sessions_removed_duplicate[col].sum() < 40:
        col_few_values.append(col)

print(len(col_few_values))
len(sessions_columns_verify)

In [34]:
sessions_removed_duplicate = sessions_removed_duplicate.drop(col_few_values, axis="columns")
sessions_removed_duplicate

## Dando merge pro dataset de treino e de teste

In [36]:
train_time_sessions = pd.merge(train_corrected_nans, sessions_removed_duplicate, how ='left', on = 'id')
#train_time_sessions

In [37]:
train_time_sessions

In [38]:
test_timed_sessions = pd.merge(test_corrected_nans, sessions_removed_duplicate, how = 'left', on = 'id')
#test_timed_sessions

## Separate lines with NAN: Remove from merge dataset and add to another

#### Colocando os registros com NAN em outro dataset

In [39]:
train_nan_sessions = train_time_sessions.loc[train_time_sessions["Chromebook"].apply(lambda x: math.isnan(x))]
original_cols = list(train_corrected_nans.columns)
train_nan_sessions = train_nan_sessions[original_cols]
train_nan_sessions

#### Removendo os registros com NAN de train_time_sessions

In [40]:
train_time_sessions = train_time_sessions.loc[train_time_sessions["Chromebook"].apply(lambda x: not math.isnan(x))]
train_time_sessions.index = range(len(train_time_sessions))

In [41]:
test_timed_sessions = test_timed_sessions.fillna(0)

In [42]:
test_timed_sessions

In [43]:
train_time_sessions.to_csv("./train_time_sessions.csv")
test_timed_sessions.to_csv("./test_timed_sessions.csv")
train_nan_sessions.to_csv("./train_nan_sessions.csv")

In [44]:
train_time_sessions = pd.read_csv("./train_time_sessions.csv")
test_timed_sessions = pd.read_csv("./test_timed_sessions.csv")
train_nan_sessions = pd.read_csv("./train_nan_sessions.csv")

In [45]:
train_time_sessions = train_time_sessions.drop('Unnamed: 0', axis=1)
test_timed_sessions = test_timed_sessions.drop('Unnamed: 0', axis=1)
train_nan_sessions = train_nan_sessions.drop('Unnamed: 0', axis=1)

## Preenchendo valores NAN de **total_secs**, **count_actions** e **signup_flow** com a mediana 

In [46]:
columns_to_fill = ["count_actions", "signup_flow"]

# Train
imp_mean = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp_mean.fit(train_time_sessions[columns_to_fill])
train_time_sessions[columns_to_fill] = imp_mean.transform(train_time_sessions[columns_to_fill])

# Test
imp_mean = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp_mean.fit(test_timed_sessions[columns_to_fill])
test_timed_sessions[columns_to_fill] = imp_mean.transform(test_timed_sessions[columns_to_fill])

columns_to_fill = ["total_secs"]

# Train
imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
imp_mean.fit(train_time_sessions[columns_to_fill])
train_time_sessions[columns_to_fill] = imp_mean.transform(train_time_sessions[columns_to_fill])

# Test
imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
imp_mean.fit(test_timed_sessions[columns_to_fill])
test_timed_sessions[columns_to_fill] = imp_mean.transform(test_timed_sessions[columns_to_fill])

train_time_sessions

### Dropando e guardando os ids

In [47]:
id_train = train_time_sessions["id"]
id_test = test_timed_sessions["id"]
train_time_sessions = train_time_sessions.drop(["id"], axis="columns")
test_timed_sessions = test_timed_sessions.drop(["id"], axis="columns")

# Análise de correlação

In [48]:
columns_list = list(train_corrected_nans.columns)
columns_list.remove("id")
columns_list

In [49]:
nominal.associations(train_time_sessions[columns_list],figsize=(15,8),mark_columns=True);

- Tirar **first_device_type**, **signup_app**, **affiliate_provider**, **first_affiliate_tracked**

In [50]:
train_time_sessions = train_time_sessions.drop(["first_device_type", "signup_app", "affiliate_provider", "first_affiliate_tracked"], axis=1)

In [51]:
test_timed_sessions = test_timed_sessions.drop(["first_device_type", "signup_app", "affiliate_provider", "first_affiliate_tracked"], axis=1)

## **PREPROCESSING DATA**

## Dropando variável alvo

In [52]:
train_Y = train_time_sessions["country_destination"]
train_Y.columns = ["country_destination"]
train_Y = pd.DataFrame(train_Y)
train_X = train_time_sessions.drop("country_destination", axis="columns")
test_X = test_timed_sessions

## Normalizando as colunas numéricas: **total_secs**, **count_actions** e **signup_flow**

### StandardScaler()

In [53]:
numerical_cols = ["age", "signup_flow", "count_actions", "total_secs"]

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() 
train_numerical_scaled = pd.DataFrame(scaler.fit_transform(train_X[numerical_cols].values))
test_numerical_scaled = pd.DataFrame(scaler.transform(test_X[numerical_cols].values))

train_X = train_X.drop(numerical_cols, axis=1)
test_X = test_X.drop(numerical_cols, axis=1)

train_numerical_scaled.columns = numerical_cols
test_numerical_scaled.columns = numerical_cols

train_X = pd.concat([train_numerical_scaled, train_X], axis=1)
test_X = pd.concat([test_numerical_scaled, test_X], axis=1)

train_X
test_X

## Aplicando ONE-HOT-ENCODING

In [54]:
object_cols = [col for col in train_X.columns if train_X[col].dtype == "object"]
object_cols

In [55]:
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_train_X = pd.DataFrame(OH_encoder.fit_transform(train_X[object_cols]))
OH_test = pd.DataFrame(OH_encoder.transform(test_X[object_cols]))

OH_train_X.index = train_X.index
OH_test.index = test_X.index

numerical_train_X = train_X.drop(object_cols, axis=1)
numerical_test = test_X.drop(object_cols, axis=1)

OH_train_X = pd.concat([numerical_train_X, OH_train_X], axis=1)
OH_test = pd.concat([numerical_test, OH_test], axis=1)

OH_train_X

### Aplicando na variável alvo

In [56]:
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_train_Y = pd.DataFrame(OH_encoder.fit_transform(train_Y))
OH_train_Y.index = train_Y.index
OH_train_Y

In [57]:
countries = train_Y["country_destination"][:10000]
for i in range(10000):
    for j in range(len(OH_train_Y.columns)):
        if OH_train_Y.iloc[i, j]:
            OH_train_Y.rename(columns = {OH_train_Y.iloc[:, j].name: countries[i]}, inplace=True)

OH_train_Y    

# Treinamento 

In [58]:
X_train, X_valid, y_train, y_valid = train_test_split(
    OH_train_X, 
    OH_train_Y, 
    train_size=0.8, 
    test_size=0.2,
    random_state=0
)

In [59]:
def build_model(table):
    model = Sequential([
        Dense(2*len(table.columns), input_dim=len(table.columns), activation="tanh"),    
        Dense(len(table.columns), activation='tanh'), 
        Dense(len(table.columns)/3, activation='tanh'), 
        Dense(12, activation='softmax'), 
    ])
    return model

model = build_model(OH_train_X)

In [60]:
model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [61]:
history = model.fit(OH_train_X.to_numpy(), OH_train_Y.to_numpy(), epochs=70, batch_size=2000, validation_split = 0.3)

## Plotando curvas de treinamento

In [62]:
def plot_history(hist):
    acc = hist.history['accuracy']
    val_acc = hist.history['val_accuracy']
    loss = hist.history['loss']
    val_loss = hist.history['val_loss']

    import matplotlib.pyplot as plt

    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, label='Training acc')
    plt.plot(epochs, val_acc, label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, label='Training loss')
    plt.plot(epochs, val_loss, label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

plot_history(history)

## Melhor resultado

In [63]:
best_score = max(history.history['val_accuracy'])

print(best_score)

# Gerando resultado para submissão

In [64]:
y_pred = model.predict(OH_test)

In [65]:
y_ids = []
countries = []
y_pred = pd.DataFrame(y_pred)
y_pred.columns = OH_train_Y.columns

for i in range(len(y_pred)):
    higher = 0
    probs = []
    y_id = id_test[i]
    probs = list(y_pred.iloc[i, :])
    for times in range(5):
        country = y_pred.iloc[:, probs.index(max(probs))].name
        probs[probs.index(max(probs))] = 0
        y_ids.append(y_id)
        countries.append(country)

In [66]:
dicio = {
    'id': y_ids,
    'country': countries
}

submission = pd.DataFrame(dicio)
submission

In [67]:
#submission.to_csv('sub.csv',index=False)

# Avaliando o  XGBClassifier

In [68]:
def set_higher_to_one(y_predicted):
    for line in range(len(y_predicted)):
        higher = 0
        for col in range(len(y_predicted.iloc[0])):
            value = y_predicted.iloc[line, col]
            if value > higher:
                higher = value
                higher_col = col
                
            y_predicted.iloc[line, col] = 0
        y_predicted.iloc[line, higher_col] = 1
    return y_predicted

def get_accuracy(y_predicted, y_validation):
    count = 0
    for i in range(len(y_validation)):
        if list(y_predicted.iloc[i, :]) == list(y_validation.iloc[i, :]):
            count += 1
    print(count)
    print(len(y_validation))
    acc = count/len(y_validation)
    print("Acc =", acc)
    return acc

In [69]:
gb = XGBClassifier(random_state=0, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='mlogloss', gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, n_estimators=100, n_jobs=-1,
                       reg_lambda=1, subsample=1)
gb.fit(X_train,y_train)

In [70]:
pred_y = pd.DataFrame(gb.predict_proba(X_valid))
pred_y

In [72]:
y_valid.index = range(len(y_valid))
y_pred_one = set_higher_to_one(pred_y.copy())
acc = get_accuracy(y_pred_one, y_valid)
acc

In [73]:
gb = XGBClassifier(random_state=0, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='mlogloss', gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, n_estimators=100, n_jobs=-1,
                       reg_lambda=1, subsample=1)
gb.fit(OH_train_X,OH_train_Y)

# Gerando resultado para submissão

In [75]:
y_pred = gb.predict_proba(OH_test)
y_pred

In [76]:
y_ids = []
countries = []
y_pred = pd.DataFrame(y_pred)
y_pred.columns = OH_train_Y.columns

for i in range(len(y_pred)):
    higher = 0
    probs = []
    y_id = id_test[i]
    probs = list(y_pred.iloc[i, :])
    for times in range(5):
        country = y_pred.iloc[:, probs.index(max(probs))].name
        probs[probs.index(max(probs))] = 0
        y_ids.append(y_id)
        countries.append(country)

In [77]:
dicio = {
    'id': y_ids,
    'country': countries
}

submission = pd.DataFrame(dicio)
submission

In [78]:
#submission.to_csv('sub.csv',index=False)

# Validação Cruzada na MLP

In [79]:
from sklearn.model_selection import KFold

histories = []
kf = KFold(n_splits=10,shuffle=True)
count = 1
for i_train, i_val in kf.split(OH_train_X):
    print('Fold:',count)
    x_treino = OH_train_X.to_numpy()[i_train]
    y_treino = OH_train_Y.to_numpy()[i_train]
    x_val = OH_train_X.to_numpy()[i_val]
    y_val = OH_train_Y.to_numpy()[i_val]
    
    model = build_model(OH_train_X)
    
    model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    history = model.fit(
        OH_train_X.to_numpy(), OH_train_Y.to_numpy(), epochs=70, batch_size=2000,
        validation_data=(x_val, y_val),
        #validation_steps=None,
        validation_batch_size=2000,
        validation_freq=list(range(100)),
        max_queue_size=10,
    )
    histories.append(history)
    count += 1
    
    plot_history(history)

In [85]:
for i in range(len(histories)):
    histories[i] = max(histories[i].history['val_accuracy'])
print("Validação cruzada =", sum(histories)/len(histories))